In [201]:
from spacy.en import English
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import yaml
from fuzzywuzzy import fuzz
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities
import string

bookname = {"title": "Harry Potter and the Philosopher's Stone", "tag": "book1"}
#bookname = "Harry Potter and the Chamber of Secrets"
parser = English()

In [125]:
# Get chapter titles from wikipedia page
#url = 'http://harrypotter.wikia.com'
#response = requests.get(url)
#page_source = response.text
#soup = BeautifulSoup(page_source, 'html5lib')
#soup(class_="wds-tabs__tab")
#series_path = soup.select("a.wds-dropdown-level-2__toggle")[0]['href']

#response = requests.get(url+series_path)
#page_source = response.text
#soup = BeautifulSoup(page_source, 'html5lib')
#book_path = soup.find("a", title=bookname)['href']

#response = requests.get(url+book_path)
#page_source = response.text
#soup = BeautifulSoup(page_source, 'html5lib')
#print(soup)
#chapter_title = soup.findAll("span", {"class": "mw-headline", "id":re.compile("Chapter*")})
#clean_title = []
#chapter_title[1].findChildren()[0]
#clean_title = [chapter_title[i].findChildren()[0]["title"] for i in range(len(chapter_title))]
#print(clean_title)

# Easier to just type it out..................
f = open('./chapters.yml')
chapters = yaml.load(f)[bookname['tag']]
print(chapters)

{1: 'THE BOY WHO LIVED', 2: 'THE VANISHING GLASS', 3: 'THE LETTERS FROM NO ONE', 4: 'THE KEEPER OF THE KEYS', 5: 'DIAGON ALLEY', 6: 'THE JOURNEY FROM PLATFORM NINE AND THREE QUARTERS', 7: 'THE SORTING HAT', 8: 'THE POTIONS MASTER', 9: 'THE MIDNIGHT DUEL', 10: "HALLOWE'EN", 11: 'QUIDDITCH', 12: 'THE MIRROR OF ERISED', 13: 'NICOLAS FLAMEL', 14: 'NORBERT THE NORWEGIAN RIDGEBACK', 15: 'THE FORBIDDEN FOREST', 16: 'THROUGH THE TRAPDOOR', 17: 'THE MAN WITH TWO FACES', 18: 'END OF THE BOOK'}


In [171]:
# Parse the book
f = open("./book1.txt")
book = dict()
chapter_list = [chapters[i] for i in chapters.keys()]
chapter_number = 0
book[chapter_number] = dict()
book[chapter_number]["title"] = bookname["title"]
c = str()
for l in f:
    l =str(l.strip())
    if fuzz.ratio(l.lower(), chapter_list[0].lower()) > 70:
        # new chapter
        print(l)
        chapter_number += 1
        chapter_list = chapter_list[1:]
        book[chapter_number] = dict()
        book[chapter_number]["title"] = l
        c = str()
    elif "CHAPTER" in l:
        # strip out chapter headings
        pass
    else:
        c += l
        book[chapter_number]["text"] = c
book[chapter_number] = {"title": l, "text":c}
print(book.keys())
# all you have to do to parse text is this:
#note: the first time you run spaCy in a file it takes a little while to load up its modules
#parsedData = parser(book[0])

THE BOY WHO LIVED
THE VANISHING GLASS
THE LETTERS FROM NO ONE
THE KEEPER OF THE KEYS
DIAGON ALLEY
THE JOURNEY FROM PLATFORM NINE AND THREE-QUARTERS
THE SORTING HAT
THE POTIONS MASTER
THE MIDNIGHT DUEL
HALLOWEEN
QUIDDITCH
THE MIRROR OF ERISED
NICOLAS FLAMEL
NORBERT THE NORWEGIAN RIDGEBACK
THE FORIBIDDEN FOREST
THROUGH THE TRAPDOOR
THE MAN WITH TWO FACES It was Quirrell.
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])


In [127]:
# cluster chapter 7 : sorting hat (it's the only thing I really remember...)
chap_entry = book[7]
print("title:", chap_entry['title'])
txt = chap_entry['text']

parsedData = parser(txt)
# Get list of characters
PERSON_list = []
for token in parsedData:
    if token.ent_type_=="PERSON":
        PERSON_list.append(str(token))
# Filter for unique name
PERSON_list = np.unique(PERSON_list)
# Filter for proper nouns without punctuations (ie. Harry's)
P_list = []
for i in PERSON_list:
    if re.match("^[a-zA-Z]*$", i) and i[0].isupper():
        P_list.append(i)
    else: pass
print(P_list)

title: THE SORTING HAT
['Albus', 'Algie', 'Anne', 'Aunt', 'Blaise', 'Blocks', 'Bloody', 'Blubber', 'Brocklehurst', 'Brown', 'Crabbe', 'Dudley', 'Dumbledore', 'Dursleys', 'Filch', 'Finnigan', 'Fred', 'Granger', 'Gryffindor', 'Hagrid', 'Hannah', 'Harry', 'Headless', 'Hermione', 'Hogwarts', 'Hufflepuffis', 'Lisa', 'Longbottom', 'MacDougal', 'Madam', 'Malfoy', 'Mandy', 'McGonagall', 'Millicent', 'Mimsy', 'Moon', 'Muggle', 'Nearly', 'Neville', 'Nicholas', 'Nitwit', 'Nott', 'Parkinson', 'Pearly', 'Percy', 'Petunia', 'Porpington', 'Potter', 'Prefect', 'Quirrell', 'Quirtell', 'Ravenclaw', 'Ron', 'Sally', 'Scabbers', 'Seamus', 'Sir', 'Slytherin', 'Snape', 'Susan', 'Terry', 'Thomas', 'Weasley', 'Zabini']


In [243]:
# Use gensim to parse relationships
# list of stop words
stoplist = set('for a of the and to in'.split())
# split for sentences
parsedData = parser(txt)
sents = []
for span in parsedData.sents:
    sent = ''.join(parsedData[i].string for i in range(span.start, span.end)).strip()
    sents.append(sent)
print(len(sents))
# filter stoplist
#ftxt = [[word for word in i.lower().split() if word not in stoplist] for i in sents]
ftxt = [[word for word in i.lower().split()] for i in sents]
# filter punctuation
#ftxt = [word for word in ftxt if re.match("^[a-zA-Z]*$", word)]
translator = str.maketrans('', '', string.punctuation)
ftxt = [[word.translate(translator) for word in sent] for sent in ftxt]
print(ftxt)
# filter repeated words
utxt = np.unique([i for j in ftxt for i in j])
# count frequency of texts
txt_count = dict()
for k in utxt:
    txt_count[k] = 0
for sent in ftxt:
    for word in sent:
        if word in utxt:
            txt_count[word] += 1
# get important words
key_txt = [[i for i in sent if txt_count[i]>1] for sent in ftxt]
#print(key_dict['gryffindor'])
dictionary = corpora.Dictionary(key_txt) #each sentence is an array..
#print(dictionary.token2id)
corpus = [dictionary.doc2bow(sent) for sent in ftxt]

# So here the text is successfully vectorized

2017-07-21 12:39:28,801 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-07-21 12:39:28,812 : INFO : built Dictionary(507 unique tokens: ['the', 'door', 'swung', 'open', 'at']...) from 234 documents (total 3616 corpus positions)


234
[['the', 'door', 'swung', 'open', 'at', 'once'], ['a', 'tall', 'blackhaired', 'witch', 'in', 'emeraldgreen', 'robes', 'stood', 'there'], ['she', 'had', 'a', 'very', 'stern', 'face', 'and', 'harrys', 'first', 'thought', 'was', 'that', 'this', 'was', 'not', 'someone', 'to', 'crossthe', 'firs', 'years', 'professor', 'mcgonagall', 'said', 'hagridthank', 'you', 'hagrid'], ['i', 'will', 'take', 'them', 'from', 'hereshe', 'pulled', 'the', 'door', 'wide'], ['the', 'entrance', 'hall', 'was', 'so', 'big', 'you', 'could', 'have', 'fit', 'the', 'whole', 'of', 'the', 'dursleys', 'house', 'in', 'it'], ['the', 'stone', 'walls', 'were', 'lit', 'with', 'flaming', 'torches', 'like', 'the', 'ones', 'at', 'gringotts', 'the', 'ceiling', 'was', 'too', 'high', 'to', 'make', 'out', 'and', 'a', 'magnificent', 'marble', 'staircase', 'facing', 'them', 'led', 'to', 'the', 'upper', 'floors'], ['they', 'followed', 'professor', 'mcgonagall', 'across', 'the', 'flagged', 'stone', 'floor'], ['harry', 'could', 'hear

In [213]:
# Use vectorized text
tfidf = models.TfidfModel(corpus, normalize=True) # step 1 -- initialize a model

2017-07-21 12:16:36,518 : INFO : collecting document frequencies
2017-07-21 12:16:36,521 : INFO : PROGRESS: processing document #0
2017-07-21 12:16:36,525 : INFO : calculating IDF weights for 234 documents and 506 features (3155 matrix non-zeros)


In [253]:
# apply transformations
corpus_tfidf = tfidf[corpus]
tfidf_sum = []
for idx,doc in enumerate(corpus_tfidf):
    i = 0
    for word in doc:
        i += word[1]
    tfidf_sum.append([idx,i])
#print(np.array(tfidf_sum))
idx_sort = np.argsort(np.array(tfidf_sum),axis=0)
tfidf_sent = np.array(tfidf_sum)[idx_sort[:,1]][::-1]
sent_idx = tfidf_sent[:,0]
sent_idx = [int(i) for i in sent_idx]
#for i in sent_idx:
#    print(sents[int(i)])
#    print('\n')

# work with top 20% of sentences
key_sent = np.array(sents)[np.array(sent_idx)[:int(len(sent_idx)*0.8)]]
print(key_sent)

[ 'Great Uncle Algie was so pleased he bought me my toad."On Harry\'s other side, Percy Weasley and Hermione were talking about lessons ("I do hope they start right away, there\'s so much to learn, I\'m particularly interested in Transfiguration, you know, turning something into something else, of course, it\'s supposed to be very difficult-"; "You\'ll be starting small, just matches into needles and that sort of thing -- ").Harry, who was starting to feel warm and sleepy, looked up at the High Table again.'
 'Dumbledore gave his wand a little flick, as if he was trying to get a fly off the end, and a long golden ribbon flew out of it, which rose high above the tables and twisted itself, snakelike, into words."Everyone pick their favorite tune," said Dumbledore, "and off we go!"And the school bellowed: "Hogwarts, Hogwarts, Hoggy Warty Hogwarts, Teach us something please, Whether we be old and bald Or young with scabby knees, Our heads could do with filling With some interesting stuff, 

In [493]:
# sentiment analysis on key sentences
# read dictionary of rated words
df = pd.read_csv('./Senti_clean.txt', sep="\s+", header=0, names=['pos','id','posscore','negscore', 'word'], usecols=[0,1,2,3,4])
sentidf = df.drop_duplicates('word')

In [525]:
#print(key_sent[0].split())
translator = str.maketrans('', '', string.punctuation)
sentiscore = []
for sent in key_sent:
    sent_sentiscore = 0
    for token in sent.split():
        q = sentidf.query("word=='%s'" %(token.translate(translator)))
        if len(q['id'].values)==1:
            sent_sentiscore += q['posscore'].values[0]-q['negscore'].values[0]
    sentiscore.append(sent_sentiscore)
# score happiness of different sentences
print(sentiscore)

[3.5, -1.125, -1.0, -2.25, 0.5, -1.625, 1.25, -0.625, 4.5, -0.25, 0.375, 1.25, 1.5, -0.375, 2.0, 0.0, 0.625, -0.625, -0.25, -0.375, -0.75, -0.625, -0.625, 0.25, 0.25, 0.25, -2.0, 0.125, 0.25, -0.375, -1.0, 0.0, 1.0, -0.375, -0.125, 1.25, -0.5, -0.25, 0.375, 0.0, -0.5, 0.25, 1.625, 0.75, -1.25, -0.5, 0.625, 0.0, 0.875, 1.375, -0.25, 0.625, -0.375, 0.375, -0.5, 0.5, 0.75, -0.375, -1.0, -0.5, -2.375, 0.25, 0.125, 0.25, 1.0, 1.125, -1.25, 0.375, -1.125, -0.625, -1.5, -1.125, 0.25, 0.125, -1.0, 0.625, -1.25, 0.375, 0.25, -0.125, -1.125, 0.125, 1.0, -0.125, -0.125, 0.125, 0.0, -0.5, -0.375, 0.0, 0.5, 0.375, 0.25, 0.25, 0.875, 0.125, -0.25, -0.625, 0.0, 0.625, 0.375, 0.25, 0.125, -0.5, 0.25, 0.125, -0.125, -1.0, 1.0, -0.125, 0.0, -0.125, 0.0, 0.125, 1.125, -0.5, 0.25, 0.25, 0.0, 0.0, 0.75, 0.125, 0.0, -0.5, -0.125, 0.0, 0.0, -0.625, -0.625, 2.125, -0.125, -0.375, -0.375, 0.0, -0.875, 2.875, 0.375, -0.25, 0.5, -0.25, -0.25, -0.5, -0.125, -0.5, -0.125, 0.125, -0.125, 0.75, -0.375, 0.25, -0.75, 

In [532]:
sentiscore_argsort = np.argsort(sentiscore)
ranked_sent = np.array(key_sent)[sentiscore_argsort][::-1]
ranked_senti = np.array(sentiscore)[sentiscore_argsort][::-1]
for i,sent in enumerate(ranked_sent):
    print(ranked_senti[i])
    print(sent)
    print('\n')

4.5
You might belong in Gryffindor, Where dwell the brave at heart, Their daring, nerve, and chivalry Set Gryffindors apart; You might belong in Hufflepuff, Where they are just and loyal, Those patient Hufflepuffis are true And unafraid of toil; Or yet in wise old Ravenclaw, if you've a ready mind, Where those of wit and learning, Will always find their kind; Or perhaps in Slytherin You'll make your real friends,


3.5
Great Uncle Algie was so pleased he bought me my toad."On Harry's other side, Percy Weasley and Hermione were talking about lessons ("I do hope they start right away, there's so much to learn, I'm particularly interested in Transfiguration, you know, turning something into something else, of course, it's supposed to be very difficult-"; "You'll be starting small, just matches into needles and that sort of thing -- ").Harry, who was starting to feel warm and sleepy, looked up at the High Table again.


2.875
There's talent, A my goodness, yes -- and a nice thirst to prove

In [34]:
# Below are some scrapy tutorials


# Let's look at the tokens
# All you have to do is iterate through the parsedData
# Each token is an object with lots of different properties
# A property with an underscore at the end returns the string representation
# while a property without the underscore returns an index (int) into spaCy's vocabulary
# The probability estimate is based on counts from a 3 billion word
# corpus, smoothed using the Simple Good-Turing method.
for i, token in enumerate(parsedData):
    print("original:", token.orth, token.orth_)
    print("lowercased:", token.lower, token.lower_)
    print("lemma:", token.lemma, token.lemma_)
    print("shape:", token.shape, token.shape_)
    print("prefix:", token.prefix, token.prefix_)
    print("suffix:", token.suffix, token.suffix_)
    print("log probability:", token.prob)
    print("Brown cluster id:", token.cluster)
    print("----------------------------------------")
    if i > 1:
        break

original: 566 The
lowercased: 501 the
lemma: 501 the
shape: 567 Xxx
prefix: 568 T
suffix: 566 The
log probability: -5.774222373962402
Brown cluster id: 30
----------------------------------------
original: 2440 door
lowercased: 2440 door
lemma: 2440 door
shape: 515 xxxx
prefix: 545 d
suffix: 1196 oor
log probability: -9.74691390991211
Brown cluster id: 7973
----------------------------------------
original: 128632 swung
lowercased: 128632 swung
lemma: 4083 swing
shape: 515 xxxx
prefix: 600 s
suffix: 1822 ung
log probability: -19.579313278198242
Brown cluster id: 8106
----------------------------------------


In [35]:
# Let's look at the sentences
sents = []
# the "sents" property returns spans
# spans have indices into the original string
# where each index value represents a token
for span in parsedData.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(parsedData[i].string for i in range(span.start, span.end)).strip()
    sents.append(sent)

for sentence in sents:
    print(sentence)

The door swung open at once.
A tall, black-hairedwitch in emerald-green robes stood there.
She had avery stern face and Harry’s first thought was that thiswas not someone to cross.“The firs’ years, Professor McGonagall,” said Hagrid.“Thank you, Hagrid
. I will take them from here.”She pulled the door wide.
The entrance hall was sobig you could have fit the whole of the Dursleys’house in it.
The stone walls were lit with flamingtorches like the ones at Gringotts, the ceiling was toohigh to make out, and a magnificent marble staircasefacing them led to the upper floors.
They followed Professor McGonagall across theflagged stone floor.
Harry could hear the drone ofhundreds of voices from a doorway to the right — therest of the school must already be here — butProfessor McGonagall showed the first years into asmall, empty chamber off the hall.
They crowded in,standing rather closer together than they wouldusually have done, peering about nervously.“Welcome to Hogwarts,” said Professor McGo

In [37]:
# Let's look at the part of speech tags of the first sentence
for span in parsedData.sents:
    sent = [parsedData[i] for i in range(span.start, span.end)]
    break

for token in sent:
    print(token.orth_, token.pos_)

The DET
door NOUN
swung VERB
open ADJ
at ADP
once ADV
. PUNCT


In [38]:
# Let's look at the dependencies of this example:
for token in parsedData:
    print(token.orth_, token.dep_, token.head.orth_, [t.orth_ for t in token.lefts], [t.orth_ for t in token.rights])

The det door [] []
door nsubj swung ['The'] []
swung ROOT swung ['door'] ['open', 'at', 'once', '.']
open dobj swung [] []
at prep swung [] []
once advmod swung [] []
. punct swung [] []
A det tall [] []
tall nsubj stood ['A'] []
, punct stood [] []
black amod hairedwitch [] []
- punct hairedwitch [] []
hairedwitch nsubj stood ['black', '-'] ['in']
in prep hairedwitch [] ['robes']
emerald amod green [] []
- punct green [] []
green amod robes ['emerald', '-'] []
robes pobj in ['green'] []
stood ROOT stood ['tall', ',', 'hairedwitch'] ['there', '.']
there advmod stood [] []
. punct stood [] []
She nsubj had [] []
had ROOT had ['She'] ['face', 'thought', 'said']
avery nmod face [] []
stern amod face [] []
face dobj had ['avery', 'stern'] ['and']
and cc face [] []
Harry compound ’s [] []
’s nsubj thought ['Harry'] []
first advmod thought [] []
thought conj had ['’s', 'first'] ['was', ',']
was ccomp thought [] ['someone']
that mark thiswas [] []
thiswas npadvmod someone ['that'] []
not neg 

oldHouse nsubj said ['My'] [',']
, punct oldHouse [] []
you nsubj know.”“Move [] []
know.”“Move ccomp said ['you'] ['along', 'now', ',', '”']
along prep know.”“Move [] []
now advmod know.”“Move [] []
, punct know.”“Move [] []
” punct know.”“Move [] []
said ROOT said ['oldHouse', 'know.”“Move'] ['voice', '.']
a det voice [] []
sharp amod voice [] []
voice dobj said ['a', 'sharp'] []
. punct said [] []
“ punct returned [] []
The det ’s [] []
SortingCeremony compound ’s [] []
’s nsubj returned ['The', 'SortingCeremony'] ['about']
about prep ’s [] ['to']
to prep about [] ['start.”Professor']
start.”Professor pobj to [] []
McGonagall nsubj returned [] []
had aux returned [] []
returned ROOT returned ['“', '’s', 'McGonagall', 'had'] ['.']
. punct returned [] []
One nsubjpass floated [] ['by', ',']
by prep One [] ['one']
one pobj by [] []
, punct One [] []
theghosts auxpass floated [] []
floated ccomp told ['One', 'theghosts'] ['away', 'through', 'form']
away advmod floated [] []
through prep

into prep burst [] ['applause']
applause pobj into [] []
as mark finishedits [] []
the det hat [] []
hat nsubj finishedits ['the'] []
finishedits advcl burst ['as', 'hat'] ['song']
song dobj finishedits [] []
. punct be [] []
It nsubj bowed [] []
bowed ROOT bowed ['It'] ['to', 'thenbecame', 'got', '!', '”', 'whispered']
to prep bowed [] ['each']
each pobj to [] ['of']
of prep each [] ['tables']
the det tables [] []
four nummod tables [] []
tables pobj of ['the', 'four'] ['and']
and cc tables [] []
thenbecame conj bowed [] ['still', 'again.“So']
quite advmod still [] []
still advmod thenbecame ['quite'] []
again.“So prep thenbecame [] []
we nsubjpass got [] []
’ve aux got [] []
just advmod got [] []
got ccomp bowed ['we', '’ve', 'just'] ['try']
to aux try [] []
try advcl got ['to'] ['on']
on prep try [] ['hat']
the det hat [] []
hat pobj on ['the'] []
! punct bowed [] []
” punct bowed [] []
Ron nsubj whispered [] []
whispered conj bowed ['Ron'] ['toHarry', '.', '“']
toHarry dobj whisper

voice dobj said ['a', 'small'] ['in']
in prep voice [] ['ear']
his poss ear [] []
ear pobj in ['his'] []
. punct said [] []
“ punct Difficult [] []
Difficult ROOT Difficult ['“'] ['.']
. punct Difficult [] []
Verydifficult ROOT Verydifficult [] ['.']
. punct Verydifficult [] []
Plenty dobj see [] ['of', ',']
of prep Plenty [] ['courage']
courage pobj of [] []
, punct Plenty [] []
I nsubj see [] []
see ROOT see ['Plenty', 'I'] ['.']
. punct see [] []
Not neg mindeither [] []
a det mindeither [] []
bad amod mindeither [] []
mindeither ROOT mindeither ['Not', 'a', 'bad'] ['.']
. punct mindeither [] []
There expl talent [] []
’s nummod talent [] []
talent ROOT talent ['There', '’s'] [',', 'oh', '—', 'and', 'thirst', 'prove', '.', '...']
, punct talent [] []
oh intj talent [] ['my', 'goodness', 'yes']
my intj oh [] []
goodness intj oh [] []
, punct yes [] []
yes intj oh [','] []
— punct talent [] []
and cc talent [] []
anice compound thirst [] []
thirst conj talent ['anice'] []
to aux prove

I nsubj liketo [] []
would aux liketo [] []
liketo ROOT liketo ['begin', ',', 'I', 'would'] ['say', '.']
say xcomp liketo [] ['words']
a det words [] []
few amod words [] []
words dobj say ['a', 'few'] []
. punct liketo [] []
And cc are [] []
here advmod are [] []
they nsubj are [] []
are ROOT are ['And', 'here', 'they'] [':', 'Nitwit!Blubber', '!']
: punct are [] []
Nitwit!Blubber nsubj are [] []
! punct are [] []
Oddment ROOT Oddment [] ['!']
! punct Oddment [] []
Tweak!“Thank nsubj sat [] []
you!”He nsubj sat [] []
sat ROOT sat ['Tweak!“Thank', 'you!”He'] ['down', '.']
back advmod down [] []
down advmod sat ['back'] []
. punct sat [] []
Everybody nsubj clapped [] []
clapped ROOT clapped ['Everybody'] ['and', 'cheered', '.']
and cc clapped [] []
cheered conj clapped [] []
. punct clapped [] []
Harry nsubj know [] []
did aux know [] []
n’t neg know [] []
know ccomp asked ['Harry', 'did', 'n’t'] ['mad', '?']
whether mark laugh [] []
to aux laugh [] []
laugh advcl mad ['whether', 'to'] 

Headless compound Nick [] []
Nick nsubj flipped ['Nearly', 'Headless'] []
flipped ROOT flipped ['looks', ',', 'Nick'] ['headback', 'onto', 'coughed', '“']
his poss headback [] []
headback dobj flipped ['his'] []
onto prep flipped [] ['neck']
his poss neck [] []
neck pobj onto ['his'] [',']
, punct neck [] []
coughed conj flipped [] [',', 'and', 'said']
, punct coughed [] []
and cc coughed [] []
said conj coughed [] [',']
, punct said [] []
“ punct flipped [] []
So ROOT So [] ['—', 'newGryffindors']
— punct So [] []
newGryffindors appos So [] ['!']
! punct newGryffindors [] []
I nsubj hope [] []
hope ROOT hope ['I'] ['going', '?']
you nsubj going [] []
’re compound going [] []
going ccomp hope ['you', '’re'] ['help']
to aux help [] []
help xcomp going ['to'] ['win']
us nsubj win [] []
win ccomp help ['us'] ['theHouse']
theHouse advmod win [] ['Championship']
Championship pobj theHouse [] ['year']
this det year [] []
year npadvmod Championship ['this'] []
? punct hope [] []
Gryffindors n

to prep got [] ['feet']
his poss feet [] []
feet pobj to ['his'] []
again advmod got [] []
. punct disappeared [] []
The det hall [] []
hall nsubj fell ['The'] []
fell ROOT fell ['hall'] ['silent.“Ahem', '—', 'words', 'now', '.']
silent.“Ahem dobj fell [] []
— punct fell [] []
just advmod words [] []
a quantmod few [] []
few nummod words ['a'] ['more']
more amod few [] []
words npadvmod fell ['just', 'few'] []
now advmod fell [] ['are']
that mark are [] []
we nsubj are [] []
are ccomp now ['that', 'we'] ['allfed']
allfed acomp are [] ['and', 'watered']
and cc allfed [] []
watered conj allfed [] []
. punct fell [] []
I nsubj have [] []
have ROOT have ['I'] ['togive', 'note']
a quantmod few [] []
few amod notices ['a'] []
start nmod notices [] ['-', 'of']
- punct start [] []
of prep start [] ['-']
- punct of [] []
term compound notices [] []
notices compound togive ['few', 'start', 'term'] []
togive dobj have ['notices'] ['years']
you.“First nummod years [] []
years dobj togive ['you.“Fi

people nsubj inthe ['the'] []
inthe ccomp surprised ['that', 'people'] ['portraits', 'andpointed']
portraits dobj inthe [] ['along']
along prep portraits [] ['corridors']
the det corridors [] []
corridors pobj along ['the'] []
whispered auxpass andpointed [] []
andpointed conj inthe ['whispered'] ['passed']
as mark passed [] []
they nsubj passed [] []
passed advcl andpointed ['as', 'they'] [',', 'or', 'led']
, punct passed [] []
or cc passed [] []
that mark led [] []
twice advmod led [] []
Percy nsubj led [] []
led conj passed ['that', 'twice', 'Percy'] ['doorways']
themthrough compound doorways [] []
doorways dobj led ['themthrough'] ['hidden']
hidden acl doorways [] ['behind']
behind prep hidden [] ['panels']
sliding amod panels [] []
panels pobj behind ['sliding'] ['andhanging']
andhanging acl panels [] ['tapestries']
tapestries dobj andhanging [] []
. punct Hewas [] []
They nsubj climbed [] []
climbed ROOT climbed ['They'] ['staircases', 'justwondering', 'had']
more amod staircases

Malfoy nsubj turned [] []
turned conj therewas ['struggled', '—then', 'Malfoy'] ['into']
into prep turned [] ['teacher']
the det teacher [] []
hook amod nosed [] []
- punct nosed [] []
nosed amod teacher ['hook', '-'] []
teacher pobj into ['the', 'nosed'] [',', 'Snape', 'became']
, punct teacher [] []
Snape appos teacher [] [',']
, punct Snape [] []
whose poss laugh [] []
laugh nsubj became ['whose'] []
became relcl teacher ['laugh'] ['high']
high acomp became [] ['and', 'cold']
and cc high [] []
cold conj high [] []
— punct therewas [] []
therewas conj pull [] ['burst', 'woke']
a det burst [] []
burst dobj therewas ['a'] ['of']
of prep burst [] ['light']
green amod light [] []
light pobj of ['green'] ['and']
and cc light [] []
Harry nsubj woke [] []
woke conj therewas ['Harry'] [',', 'shaking', '.']
, punct woke [] []
sweatingand advmod shaking [] []
shaking advcl woke ['sweatingand'] []
. punct woke [] []
He nsubj rolled [] []
rolled ROOT rolled ['He'] ['over', 'and', 'fell', 'rememb

In [39]:
# Let's look at the named entities of this example:
for token in parsedData:
    print(token.orth_, token.ent_type_ if token.ent_type_ != "" else "(not an entity)")

print("-------------- entities only ---------------")
# if you just want the entities and nothing else, you can do access the parsed examples "ents" property like this:
ents = list(parsedEx.ents)
for entity in ents:
    print(entity.label, entity.label_, ' '.join(t.orth_ for t in entity))

The (not an entity)
door (not an entity)
swung (not an entity)
open (not an entity)
at (not an entity)
once (not an entity)
. (not an entity)
A (not an entity)
tall (not an entity)
, (not an entity)
black (not an entity)
- (not an entity)
hairedwitch (not an entity)
in (not an entity)
emerald (not an entity)
- (not an entity)
green (not an entity)
robes (not an entity)
stood (not an entity)
there (not an entity)
. (not an entity)
She (not an entity)
had (not an entity)
avery (not an entity)
stern (not an entity)
face (not an entity)
and (not an entity)
Harry PERSON
’s PERSON
first ORDINAL
thought (not an entity)
was (not an entity)
that (not an entity)
thiswas (not an entity)
not (not an entity)
someone (not an entity)
to (not an entity)
cross.“The (not an entity)
firs’ DATE
years DATE
, (not an entity)
Professor (not an entity)
McGonagall PERSON
, (not an entity)
” (not an entity)
said (not an entity)
Hagrid.“Thank ORG
you (not an entity)
, (not an entity)
Hagrid GPE
. (not an entity)

swallowed.“How PERSON
exactly (not an entity)
do (not an entity)
they (not an entity)
sort (not an entity)
us (not an entity)
into (not an entity)
Houses (not an entity)
? (not an entity)
” (not an entity)
he (not an entity)
askedRon.“Some (not an entity)
sort (not an entity)
of (not an entity)
test (not an entity)
, (not an entity)
I (not an entity)
think (not an entity)
. (not an entity)
Fred PERSON
said (not an entity)
it (not an entity)
hurts (not an entity)
a (not an entity)
lot (not an entity)
, (not an entity)
butI (not an entity)
think (not an entity)
he (not an entity)
was (not an entity)
joking.”Harry (not an entity)
’s (not an entity)
heart (not an entity)
gave (not an entity)
a (not an entity)
horrible (not an entity)
jolt (not an entity)
. (not an entity)
A (not an entity)
test (not an entity)
? (not an entity)
In (not an entity)
front (not an entity)
ofthe (not an entity)
whole (not an entity)
school (not an entity)
? (not an entity)
But (not an entity)
he (not an entity)

the (not an entity)
sandy (not an entity)
- (not an entity)
haired (not an entity)
boynext (not an entity)
to (not an entity)
Harry PERSON
in PERSON
the (not an entity)
line (not an entity)
, (not an entity)
sat (not an entity)
on (not an entity)
the (not an entity)
stool (not an entity)
for (not an entity)
almost (not an entity)
awhole (not an entity)
minute (not an entity)
before (not an entity)
the (not an entity)
hat (not an entity)
declared (not an entity)
him (not an entity)
aGryffindor.“Granger (not an entity)
, (not an entity)
Hermione!”Hermione (not an entity)
almost (not an entity)
ran (not an entity)
to (not an entity)
the (not an entity)
stool (not an entity)
and (not an entity)
jammed (not an entity)
the (not an entity)
hateagerly (not an entity)
on (not an entity)
her (not an entity)
head.“GRYFFINDOR (not an entity)
! (not an entity)
” (not an entity)
shouted (not an entity)
the (not an entity)
hat (not an entity)
. (not an entity)
Ron PERSON
groaned (not an entity)
. (no

in (not an entity)
a (not an entity)
row (not an entity)
! (not an entity)
The (not an entity)
Bloody PERSON
Baron’sbecoming (not an entity)
almost (not an entity)
unbearable (not an entity)
— (not an entity)
he (not an entity)
’s (not an entity)
the (not an entity)
Slytheringhost.”Harry (not an entity)
looked (not an entity)
over (not an entity)
at (not an entity)
the (not an entity)
Slytherin NORP
table (not an entity)
and (not an entity)
saw (not an entity)
ahorrible (not an entity)
ghost (not an entity)
sitting (not an entity)
there (not an entity)
, (not an entity)
with (not an entity)
blank (not an entity)
staring (not an entity)
eyes (not an entity)
, (not an entity)
agaunt (not an entity)
face (not an entity)
, (not an entity)
and (not an entity)
robes (not an entity)
stained (not an entity)
with (not an entity)
silver (not an entity)
blood (not an entity)
. (not an entity)
Hewas (not an entity)
right (not an entity)
next (not an entity)
to (not an entity)
Malfoy PERSON
who (no

times (not an entity)
. (not an entity)
At (not an entity)
last (not an entity)
, (not an entity)
only (not an entity)
the (not an entity)
Weasley ORG
twins (not an entity)
were (not an entity)
left (not an entity)
singing (not an entity)
along (not an entity)
to (not an entity)
avery (not an entity)
slow (not an entity)
funeral (not an entity)
march (not an entity)
. (not an entity)
Dumbledore PERSON
conducted (not an entity)
theirlast CARDINAL
few CARDINAL
lines (not an entity)
with (not an entity)
his (not an entity)
wand (not an entity)
and (not an entity)
when (not an entity)
they (not an entity)
hadfinished (not an entity)
, (not an entity)
he (not an entity)
was (not an entity)
one CARDINAL
of (not an entity)
those (not an entity)
who (not an entity)
clapped (not an entity)
loudest.“Ah (not an entity)
, (not an entity)
music (not an entity)
, (not an entity)
” (not an entity)
he (not an entity)
said (not an entity)
, (not an entity)
wiping (not an entity)
his (not an entity)
eye

NameError: name 'parsedEx' is not defined